<a href="https://colab.research.google.com/github/wesleydecezere/programa_ds_fc/blob/master/Entrega%206%20-%20S%C3%A9ries%20Temporais/Series_Temporais_Exercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Séries temporais - Exercício</center>
___

Um dos grandes desafios das empresas é a previsão de vendas. Um bom modelo de previsão das vendas da empresa permite um melhor planejamento de gastos das empresas e da produção, permite uma estimação de lucros e até mesmo auxilia a empresa a determinar metas, avaliar o seu desempenho e ter uma melhor visão de futuro, ajudando na atração de possíveis investidores.

Esse notebook contém o exercício prático da aula de séries temporais. O *dataset* que utilizaremos e foi disponibilizado na pasta de dados da aula vem originalmente de um desafio de recrutamento do Walmart - que pode ser acessado [aqui](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data). Ele contém vendas anônimas por departamento para 45 lojas Walmart, bem como variáveis de apoio.

Assim, o exercício proposto aqui é de **prever as vendas semanais das lojas da Walmart**. Para isso, sugerimos você utilizar o modelo de machine learning *Random Forest* que vimos na aula, mas você pode também experimentar outros modelos que aprendemos.

Após todo o conhecimento que você adquiriu ao longo da trilha, você já está preparado para fazer um projeto sozinho. Assim, diferentemente dos outros exercícios, deixaremos o desafio para você estruturar o seu próprio código e modelo. Caso tenha dificuldade ou não saiba por onde começar, te recomendamos consultar os exemplos feitos ao longo das aulas para aplicar no exercício, e sinta-se livre para mandar suas dúvidas no grupo.

Bom, agora é com você!

In [ ]:
# Importando as bibliotecas principais utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
features_ = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
train = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
stores = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')
test = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
sample_submission = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')

In [ ]:
walmart = pd.read_csv('../input/walmart-sales/walmart_sales.csv')
walmart.head()

#### Exploração dos dados

##### Departamentos por loja
* O número de departamentos é diferente de uma loja para outra: a maioria tem entre 70 e 80 departamentos, mas algumas têm cerca de 60. 


In [ ]:
# número dos de departamentos
walmart_store = np.sort(walmart.Store.unique())
walmart_store

In [ ]:
dept_qtd = list()

for idx, store in enumerate(walmart_store): 
   dept_qtd.append(walmart[walmart.Store == store].Dept.unique().size)

c = plt.bar(walmart_store, dept_qtd)
plt.ylabel('Number of departamets')
plt.xlabel('Store')
plt.xticks(range(1,46,2))
plt.grid(True)
plt.show()

##### Quantidade de amostras por departamento e loja
* Em uma mesma loja, os departamentos tem números de amostras bem diferentes: enquanto alguns departamentos tem mais de 100 amostras, outros tem apenas 3.
* Em um mesmo departamento de lojas diferentes, também não há uma regularidade no número de amostras: o departamento 96 da loja 1 tem cerca de 80, enquanto o mesmo departamento da loja 45 tem exatas 2 amostras. 

In [ ]:
walmart[walmart.Store == 1].groupby('Dept')['Date'].count().plot.bar(figsize=(15,5))

In [ ]:
walmart[walmart.Store == 45].groupby('Dept')['Date'].count().plot.bar(figsize=(15,5))

##### Características iguais nos departamentos

Em uma mesma data, todos os departamentos de uma mesma loja apresentam as mesmas características (inclusive as de promoção, MarkDown1-5).


In [ ]:
walmart_store1_2010_02_2016 = walmart[(walmart.Store == 1) & (walmart.Date == '2010-12-31')]
walmart_store1_2010_02_2016.head()

In [ ]:
walmart_store1_2010_02_2016.nunique()

Portanto, dado uma data qualquer, nenhuma das características disponíveis influencia realmente o número de vendas semanais entre os departamentos de uma mesma loja. Nesta data, essas características podem influenciar somentes o volume de vendas entre lojas diferentes, mas não necessariamente. 

Isso me leva a pensar que nenhuma das características é boa para auxiliar na previsão de vendas semanais entre os departamentos de uma mesma loja. Algumas podem ser úteis, contudo, para avaliar as vendas entre departamentos de lojas diferentes.

## Exercício 1: Pré-processamento de dados e criação de Features



#### Limpeza dos dados

In [ ]:
walmart.info()

##### Tratamento de nulos

In [ ]:
walmart_markdown = walmart.loc[:, 'MarkDown1':'MarkDown5']

print('Procentagem de nulos')

for markdown in walmart_markdown.columns:
  md = walmart_markdown[markdown]
  print(markdown, ':', md.isna().sum() / md.size * 100)

Em todas as features de promoção, há mais de 60% de registros nulos. 

Conforme consta na descrição do dataset do Kaggle, os registros só estão disponíveis a partir de Nov 2011 e não não estão disponíveis para todas as lojas a todo momento.

In [ ]:
walmart.isna().sum()

In [ ]:
walmart_treated = walmart.fillna(0)
walmart_treated.isna().sum()

##### Conversões pertinentes (Data as datetime)


A variável `Date` é do tipo `object`. Portanto, cabe convertê-la para `datetime`

In [ ]:
walmart_treated['Date'] = pd.to_datetime(walmart_treated.Date)

In [ ]:
walmart_treated[['Date']].info()



##### Ordenação temporal das amostras (data como índice)

In [ ]:
walmart_treated = walmart_treated.sort_values(by='Date')

### Criação de features



#### Informação temporal

Os registros vão de 05/fev/2010 até 01/nov/2012, com frequência semanal. Como uma datetime não diz muita coisa para o modelo, devemos converter a Date para algum formato útil.

In [ ]:
walmart_treated.Date.dt.isocalendar()

A contagem de vendas semanais é fechada sempre na sexta-feira (5º dia útil da semana). Portanto, o dia das datas não nos traz informação útil. Cabe manter no dataset somentre o ano e a semana de cada registro.

In [ ]:
walmart_treated['week'] = walmart_treated.Date.dt.isocalendar().week
walmart_treated['year'] = walmart_treated.Date.dt.isocalendar().year

walmart_treated.drop('Date', axis=1, inplace=True)

In [ ]:
walmart_treated.head(2)

In [ ]:
walmart_treated.tail(2)

#### Variáveis dummy

Como temos uma variável categórica multiclasse, a Type, devemos criar variáveis dummy para cada categoria, visando utilizá-las no Random Forest.

In [ ]:
walmart_treated = pd.get_dummies(walmart_treated)
walmart_treated.head(2)

## Exercício 2: Separação dos conjuntos de treino e teste

* Split que não embaralhe os dados (`shuffle = False`)
* Expanding Windows para a validação cruzada (se não demorar uma eternidae, pois temos mais de 282 mil registros)

Primeiramente, temos que separar o target das features.

In [ ]:
X = walmart_treated.drop('Weekly_Sales', axis=1)
y = walmart_treated['Weekly_Sales']

print(X.shape)
print(y.shape)

Na sequência, fazemos o split de treino e teste, sem embaralhar as amostras.

In [ ]:
from sklearn.model_selection import train_test_split

X_training, X_test, y_training, y_test = train_test_split(X, y, 
                                                          test_size = 0.25, 
                                                          shuffle = False)

print("Train set X", X_training.shape)
print("Train set y", y_training.shape)
print("Test set X", X_test.shape)
print("Test set y", y_test.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                          test_size = 0.33, 
                                                          shuffle = False)

print("Train set X", X_train.shape)
print("Train set y", y_train.shape)
print("Validation set X", X_val.shape)
print("Validation set y", y_val.shape)

Por fim, definimos uma Expanding Windows para utilização na validação cruzada.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

cv = TimeSeriesSplit(n_splits=10)

## Exercício 3: Construção do modelo e análise dos resultados

In [ ]:
# Importando as bibliotecas que vamos utilizar no modelo
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

#### Hiperparâmetros padrão

Valor padrão dos hiperparâmetros selecionados
* `n_estimators`: 100
* `max_depth`: None
* `min_samples_split`: 2
* `min_samples_leaf`: 1 
* `max_features`: 'auto'

In [ ]:
SEED = 42

#### Cross Validation: Expanding Windows

Claramente estamos com overfit, pois, ao utilizar diferentes dados para validação, obtivemos um resultado consideravelmente pior: $R^2$ caiu de 0.945 para 0.915.

Portanto, cabe uma investigação dos melhores hiperparâmetros com o GridSearch.

#### GridSearch

Após alguns longos GridSearch, constata-se que os melhores hiperparâmetros são:
* `n_estimators`: 50
* `max_depth`: None
* `min_samples_split`: 4
* `min_samples_leaf`: 1 
* `max_features`: 14

Por isso, vamos treinar o modelo com a combinação encontrada.


In [ ]:
# seta melhor opção baseada nos hiperparâmetros
# utiliza as melhores opções de hiperparâmetros encontrados

#rf_model_cv_gs.set_params(n_estimators = grid_search.best_params_['n_estimators'],
#                          max_features = grid_search.best_params_['max_features'],
#                          min_samples_split = grid_search.best_params_['min_samples_split'],
#                          )
rf_model_cv_gs = RandomForestRegressor(random_state=SEED, n_jobs=-1)

rf_model_cv_gs.set_params(n_estimators = 50,
                          max_features = 14,
                          min_samples_split = 4,
                          )

# treina modelo com todos os dados de treino disponíveis e com os melhores hiperparâmetros encontrados
rf_model_cv_gs.fit(X_training, y_training)

#### Resultados

In [ ]:
# prevendo os valores no dataset de teste
y_pred_test = rf_model_cv_gs.predict(X_test)

In [ ]:
print("Score on validation set: {:.3f}".format(rf_model_cv_gs.score(X_test, y_test)))
print("Mean absolute error (MAE): {:.3f}".format(mean_absolute_error(y_test, y_pred_test)))
print("R² Score: {:.3f}".format(r2_score(y_test, y_pred_test)))

Anteriormente, na validação cruzada com os melhores hiperparâmetros do GridSearch, obtivemos $R^2$ médio de 0,993 e 0,919 nos sets de treino e validação, respectivamente. Estes valores formam um pouco melhores que os obtidos com os hiperparâmetros padrão.

Agora, para o dataset de treino, conseguimos resultados muito bons: $R^2$ de 0,965 e MAE de 2056,159, a qual tinha ficado em 2420.472 para o modelo com hiperparâmetros padrão e sem validação cruzada.

In [ ]:
# Desenhando o gráfico de valores previstos por valores reais
plt.figure(figsize=(18,8))
plt.title('Walmart weakly sales - Predicted vs Real',fontsize=20)

df = pd.DataFrame({'real':y_test,'RF':y_pred_test})
df = df.reset_index(drop=True)


plt.plot(df.iloc[-100:])
plt.legend(['Real weekly sales','RF Predicted weekly sales'],fontsize=20)
plt.ylabel('Weekly sales',fontsize=20)
plt.xlabel('Samples ordered by date',fontsize=20)
plt.show()

No grafico acima, para as 100 últmas amostras do dataset de teste, podemos ver como os valores previstos pelo modelo (em larajna) se comprortam em relação as valores reais (em azul).

In [ ]:
features = X.columns
importances = rf_model_cv_gs.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

No gráfico de feature importance, percebemos que a variável Dept teve uma importância ligeiramente maior, enquanto a Size teve importância ligeiramente menor, em relação ao modelo com hiperparâmetros padrão. Mesmo assim, ambas ainda foram as variáveis mais importantes para a decisão.

## Submission

In [ ]:
feat_sto = features_.merge(stores, how='inner', on='Store')

feat_sto['Date'] = pd.to_datetime(feat_sto['Date'])
feat_sto['Week'] = feat_sto.Date.dt.isocalendar().week
feat_sto['Year'] = feat_sto.Date.dt.isocalendar().year

feat_sto.head()

In [ ]:
test['Date'] = pd.to_datetime(test['Date'])

test_detail = test.merge(feat_sto, 
                           how='inner',
                           on=['Store','Date','IsHoliday']).sort_values(by=['Store',
                                                                            'Dept',
                                                                            'Date']).reset_index(drop=True)

test_detail.fillna(0, inplace=True)
test_detail.drop('Date', axis=1, inplace=True)

test_detail.head()

In [ ]:
test_final = pd.get_dummies(test_detail)
test_final.head()

In [ ]:
test_predict = rf_model_cv_gs.predict(test_final)

In [ ]:
sample_submission['Weekly_Sales'] = test_predict
sample_submission.to_csv('submission.csv',index=False)